# Tutorial - Decomposition of a single spectrum

Here we show how to prepare and decompose a single spectrum. This can be instructive in how parameter settings influence and impact the decomposition and can be helpful in optimizing the settings.

In [ ]:
import os

%matplotlib inline
import matplotlib.pyplot as plt

from gausspyplus.prepare import GaussPyPrepare
from gausspyplus.decompose import GaussPyDecompose

In [ ]:
prepare = GaussPyPrepare()
prepare.path_to_file = os.path.join(
    '..', 'gausspyplus', 'data', 'grs-test_field.fits')
prepare.p_limit = 0.02
prepare.pad_channels = 5
prepare.signal_mask = True
prepare.min_channels = 100
prepare.mask_out_ranges = []
prepare.snr = 3.
prepare.significance = 5.
prepare.snr_noise_spike = 5.
data_location = (31, 40)  # given in (ypix, xpix)
prepared_spectrum = prepare.return_single_prepared_spectrum(data_location)

decompose = GaussPyDecompose()
decompose.two_phase_decomposition = True
decompose.alpha1 = 2.58
decompose.alpha2 = 5.14

#  GaussPy+ parameters (only take effect if 'improve_fitting' is set to True)
decompose.improve_fitting = True
decompose.exclude_means_outside_channel_range = True
decompose.min_fwhm = 1.
decompose.max_fwhm = None
decompose.snr = 3.
decompose.snr_fit = None
decompose.significance = 5.
decompose.snr_negative = None
decompose.min_pvalue = 0.01
decompose.max_amp_factor = 1.1
decompose.refit_neg_res_peak = True
decompose.refit_broad = True
decompose.refit_blended = True
decompose.separation_factor = 0.8493218
decompose.fwhm_factor = 2.
decompose.single_prepared_spectrum = prepared_spectrum
decomposed_test = decompose.decompose()
                             

Running the previous cell printed some decomposition diagnostics and produced a multi-panel figure with the following four figures:

- **upper left panel**: This figure shows how GaussPy determines the initial guesses for the Gaussian fit parameters in its first decomposition phase. The original input spectrum is shown in black solid lines and the S/N threshold for the original spectrum (``snr`` parameter) is shown with the horizontal dashed black line. The red line indicates the second derivative of the spectrum that was obtained after smoothing the original spectrum with the $\alpha_{1}$ (``alpha1``) parameter and the red dashed horizontal lines show the corresponding S/N threshold for the second derivative. Initial guesses for the Gaussian fit parameters are shown in green and are determined by peaks in the original spectrum and second derivative of the smoothed spectrum.

- **upper right panel**: This figure shows the intermediate fit results (blue line) obtained with the initial guesses for the Gaussian fit parameters obtained in the first decomposition phase. Note that this step is not performed if the ``two_phase_decomposition`` parameter is set to ``False``.

- **lower left panel**: This figure shows the second decomposition phase of GaussPy, in which it searches for new peaks in the residual from the intermediate fit. This phase proceeds in a similar way to the first phase, with the difference that now the $\alpha_{2}$ (``alpha2``) parameter is used to smooth the residual before the derivatives are computed. Note that this step is not performed if the ``two_phase_decomposition`` parameter is set to ``False``.

- **lower right panel**: This figure shows the final fit results of GaussPy (orange lines). If the ``improve_fitting`` parameter is set to ``True``, the decomposition results correspond to the outcome of the improved fitting routine of GaussPy+, which aims to further improve the final fitting results of GaussPy. Users can set the ``improve_fitting`` parameter to ``True`` or ``False`` to check the effect of the improved fitting routine of GaussPy+. Note that the fit results of GaussPy+ correspond to the stage before the spatially coherent refitting phases, so decomposition results can still change later on based on neighboring fit results.

### Decomposition results

In the preparation and decomposition steps we obtain two dictionaries. Lets first take a look at the contents of the dictionary created in the preparation step:

In [ ]:
prepared_spectrum.keys()

This dictionary contains the following keys:

- **data_list**: A list containing information about the intensity values of the spectra

- **x_values**: A list containing information about the number of spectral channels per spectrum

- **error**: Information about the rms-error of the spectra

- **index**: Index of the spectrum in the dictionary (identical to the **index_fit** keyword in the dictionary containing the decomposition results)

- **location**: Location of the spectrum in the spectral cube; given as (Y position, X position)

- **header**: FITS header of the spectral cube

- **nan_mask**: Information about NaN values contained in the spectral cube

- **signal_ranges**: Regions in the spectrum determined as containing signal peaks. Goodness-of-fit criteria (such as reduced chi-square values and the AICc parameter use only spectral channels contained in these regions)

- **noise_spike_ranges**: Regions in the spectrum determined as containing negative noise spikes. These regions are masked out for goodness-of-fit calculations.

Next, lets take a look of the dictionary containing the decomposition results:

In [ ]:
from pprint import pprint
pprint(decomposed_test)

This dictionary contains the following keys:

- **index_fit**: Index in the dictionary (identical to the 'index' keyword in the dictionary containing the decomposition results)

- **amplitudes_fit**, **fwhms_fit**, **means_fit**: Lists containing the fit parameters of the N Gaussian components of the final fit solution in the form of [amplitude_1, amplitude_2, ..., amplitude_N], [fwhm_value_1, fwhm_value_2, ..., fwhm_value_N], [mean_position_value_1, mean_position_value_2, ..., mean_position_value_N]

- **amplitudes_fit_err**, **fwhms_fit_err**, **means_fit_err**: Lists containing the corresponding errors of the fit parameters

- **best_fit_rchi2**: List of the reduced chi-square values of the best fit solutions

- **best_fit_aicc**: List of the values of the corrected Akaike information criterion for the best fit solutions

- **pvalue**: List of p-values obtained from the normality tests for the normalised residuals

- **amplitudes_initial**, **fwhms_initial**, **means_initial**: Lists containing the fit parameters of the initial guesses for the components (i.e., all components identified in the upper and lower left panels in the Figure above)

- **N_components**: List containing the number of fit components of the best fit solutions

- **N_blended**: List containing the info about the number of blended components

- **N_neg_res_peak**: List containing the info about the number of negative residual peaks introduced by the fit

- **log_gplus**: List containing logs of all successful refits of the spectrum in the improved fitting routine of GaussPy+:
    - 1: new fit component added from search of positive peaks in the residual
    - 2: refit of spectrum due to the presence of negative residual peak(s) in the residual
    - 3: refit of the spectrum due to the presence of broad fit component(s)
    - 4: refit of the spectrum due to the presence of blended fit components

- **quality_control**: List containing logs of fit components by in-built quality criteria:
    - 0: FWHM value of component exceeded the ``max_fwhm`` parameter
    - 1: FWHM value of component was smaller than the ``min_fwhm`` parameter
    - 2: Amplitude value of component did not satisfy the required minimum S/N ratio (default value is ``snr``/2)
    - 3: Significance value of the fit components does not satisfy the required minimum value given by the ``significance``parameter
    - 4: Mean position of the fit component was outside the channel range
    - 5: Mean position of the fit component was outside the determined **signal_ranges** **AND** the fit component did not satisfy the significance criterion)